In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x_train = torch.tensor([
    [[[4, 5, 6, 7], [5, 6, 7, 8], [8, 9, 10, 11], [4, 6, 7, 8]]],
    [[[-4, 5, 6, -7], [5, -6, 7, 8], [-8, 9, -10, 11], [-4, -6, -7, 8]]],
]).float().to(device)

x_train.div_(8)
y_train = torch.tensor([1, 0]).float().to(device)

In [4]:
x_train.shape

torch.Size([2, 1, 4, 4])

In [5]:
y_train.shape

torch.Size([2])

In [6]:
import torch
import torch.nn as nn
from torch.optim import Adam

def get_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 2),
        nn.MaxPool2d(kernel_size = 2), # reduz as dimensoes
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1, 1),
        nn.Sigmoid() # função de ativaçao
    ).to(device)

    loss_fn = nn.BCELoss() # faz a loss function ser a binary cross entropy
    optimizer = Adam(model.parameters(), lr = 0.01) # learning rate de 0.01

    return model, loss_fn, optimizer

In [7]:
model, criterion, optimizer = get_model()

In [8]:
model

Sequential(
  (0): Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))
  (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (2): ReLU()
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=1, out_features=1, bias=True)
  (5): Sigmoid()
)

In [9]:
import torch

def train_batch(x, y, model, optimizer, loss_fn):
    model.train()

    optimizer.zero_grad()
    prediction = model(x)
    batch_loss = loss_fn(prediction.squeeze(), y.squeeze())
    batch_loss.backward()
    optimizer.step() # atualiza os parâmetros do modelo com base nos gradientes

    return batch_loss.item()

In [10]:
from torch.utils.data import DataLoader, TensorDataset, Dataset

trn_dl = DataLoader(TensorDataset(x_train, y_train))

In [12]:
for epoch in range(2000):
    for ix, batch in enumerate(trn_dl):
        x, y = batch
        x = x.to(device)
        y = y.to(device)

        loss = train_batch(x, y, model, optimizer, criterion)

In [13]:
model(x_train[:1])

tensor([[0.4999]], grad_fn=<SigmoidBackward0>)